# Modflow Post Proccessing Exercise



Use the Freyberg modflow NWT model "../data/Freyberg_transient" to answer the following questions

Assume the model units are in meters and days 

Please make figures look simple, nice and proffesional 

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import flopy
import platform
%matplotlib inline 

In [ ]:
if "window" in platform.platform().lower():
    mf_exe = os.path.join("..","bin", "win",'mfnwt.exe')
elif "linux" in platform.platform().lower():
    mf_exe = os.path.join("..","bin", "linux",'mfnwt')
elif 'mac'in platform.platform().lower():
    mf_exe = os.path.join("..","bin", "mac",'mfnwt')
mf_exe

In [ ]:
model_ws = os.path.join('..','data','Freyberg_transient')
mf = flopy.modflow.Modflow.load('freyberg.nam',model_ws=model_ws,version='mfnwt',check=False,exe_name=mf_exe)

## 1. Plot a colormap of the hydraulic conductivity in all three layers

Please show a colorbar, and label units

hint; the hydraulic conductivity is stored in the UPW package

In [ ]:
nlay = mf.dis.nlay
hk = mf.upw.hk.array
for lay in range(nlay):
    fig, ax = plt.subplots(figsize=(8,6))
    ### write your code here

    ax.set_title(f'Hydraulic Conductivity in Layer {lay+1}')
    fig.tight_layout()
    

## 2. Plot a hydrograph for each well below from 01/01/2015 - 01/01/2017

hint; heads are stored in the binary file freyberg.hds. Use the flopy function for reading such files - 
flopy.utils.HeadFile()

In [ ]:
df = pd.DataFrame({'wellid': ['MW1', 'MW2', 'MW5', 'PW6'],
                   'layer': [1, 1, 1, 1],
                   'row': [9, 24, 18, 34],
                   'column': [9, 7, 16, 12],
                   'top_screen':[10,13.5,9.825,-15],
                   'bot_screen':[7.5,10,6.425,-20]})

df

In [ ]:
hdsobj = flopy.utils.HeadFile(os.path.join(model_ws,'freyberg.hds'))
start_datetime = pd.to_datetime(mf.modeltime.start_datetime)
for i, dfrow in df.iterrows():
    l, r, c = dfrow['layer'], dfrow['row'], dfrow['column']

    fig, ax = plt.subplots(figsize=(8,6))
    ### write your code here

    
    ###
    ax.set_title(dfrow['wellid'])
    fig.tight_layout()

## 3.a Modify the pumping well in the wellfile and run the model

Increase pumping at the PW6 well by 25% for all stress periods then rewrite the model in this directory; '../data/Freyberg_transient_25'

well = PW6

layer =1

row = 34

column = 12


In [ ]:
model_ws2 = os.path.join('..', 'data', 'Freyberg_transient_25')
mf2 = flopy.modflow.Modflow.load('freyberg.nam',model_ws=model_ws,version='mfnwt',check=False,exe_name=mf_exe)
wel = mf2.wel
spd = wel.stress_period_data.data
spd

In [ ]:

# hint; modify the wel stress period data (spd) by increaseing the flux by 25% for PW6
nper = mf2.dis.nper
for sp in range(nper):
    ### write your code here
    spd[sp]


    ###
    
wel2 = flopy.modflow.ModflowWel(mf2,stress_period_data=spd,dtype=spd[0].dtype)

In [ ]:
mf2.change_model_ws(model_ws2)
mf2.write_input()
mf2.run_model()

## 3.B Calculate the drawdown caused by the increase in pumping in all wells and show plot

In [ ]:
hdsobj = flopy.utils.HeadFile(os.path.join(model_ws,'freyberg.hds'))
hdsobj2 = flopy.utils.HeadFile(os.path.join(model_ws2,'freyberg.hds'))

start_datetime = pd.to_datetime(mf.modeltime.start_datetime)
for i, dfrow in df.iterrows():
    l, r, c = dfrow['layer'], dfrow['row'], dfrow['column']

    fig, ax = plt.subplots(figsize=(8,6))
    ### write your code here


    ###
    ax.set_title(f"Drawdown in {dfrow['wellid']}")
    ax.set_xlim([pd.to_datetime('01/01/2015'),pd.to_datetime('01/01/2016')])
    fig.tight_layout()